In [1]:
import os
from datasets import load_dataset, DatasetDict, load_from_disk
from transformers import (
    AutoTokenizer, 
    AutoModelForSequenceClassification, 
    TrainingArguments, 
    Trainer,
    DataCollatorWithPadding,
    set_seed
)
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
import numpy as np


C:\Users\Jeeva M\AppData\Roaming\Python\Python313\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
set_seed(42)

dataset_path = "../data/tokenized_dataset"
if os.path.exists(dataset_path):
    dataset = load_from_disk(dataset_path)
    print("✅ Tokenized dataset loaded from disk.")
else:
    raise FileNotFoundError("❌ Tokenized dataset not found. Run data_preprocessing.py first.")

In [ ]:
model_ckpt = "bert-base-multilingual-cased"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
model = AutoModelForSequenceClassification.from_pretrained(model_ckpt, num_labels=3)

In [ ]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    preds = np.argmax(predictions, axis=1)
    return {
        "accuracy": accuracy_score(labels, preds),
        "f1": f1_score(labels, preds, average="weighted"),
        "precision": precision_score(labels, preds, average="weighted"),
        "recall": recall_score(labels, preds, average="weighted"),
    }

In [ ]:
training_args = TrainingArguments(
    output_dir="../models/trained_model",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    logging_dir="../models/logs",
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    logging_steps=10,
    do_eval=True
)

In [ ]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)


In [ ]:
trainer.train()

trainer.save_model("../models/trained_model")
tokenizer.save_pretrained("../models/tokenizer")


In [ ]:
results = trainer.evaluate(dataset["test"])
print("Test Results:", results)